# 04 Data Encoding

## Notebook Overview

This notebook transforms categorical time features into a machine-learning-friendly format using one-hot encoding.

**Key Steps:**

* **Input:** Cleaned hourly-level dataset (`data_cleaned.csv`)
* **Encoding:** One-hot encodes `weekday` and `timing` columns to capture cyclical and behavioral patterns across time
* **Output:** Saves the encoded dataset as `data_encoded.csv` for use in model training

> Purpose: Ensure temporal categorical features are numerically represented without introducing implicit ordering.

### Thoughts, Tradeoffs & Considerations

* **Avoided ordinal encoding:** `weekday` and `timing` may look ordered but aren’t numerically meaningful (e.g., Friday ≠ 5). One-hot encoding avoids injecting false structure into tree models.
* **Kept full dummies (no drop-first):** Didn’t drop the first column in one-hot encoding. For trees, multicollinearity isn’t a problem, and interpretability is better with all categories visible.
* **Didn’t use cyclical encoding:** Considered sine/cosine transformations for `hour`, but since we already captured behavior via `timing`, it was redundant here.
* **Prefix naming:** Added `wd_` and `time_` prefixes to avoid name clashes or confusion when inspecting feature importance later.
* **Sparse matrix not needed:** Although one-hot expands columns, the dataset is still small enough to keep in dense format without performance issues.

> Encoding time-based context like weekday and timing helps models capture behavioral patterns (e.g., higher energy use on Mondays or in the evening). Clean, explicit encoding here will help with interpretability and downstream feature selection.

In [1]:
import pandas as pd

In [2]:
# Show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# widen the column width and overall display width
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 0)

In [3]:
df: pd.DataFrame = pd.read_csv('../data/interim/data_cleaned.csv')

In [4]:
# One-hot encode 'weekday' and 'timing'
df_encoded = pd.get_dummies(df, columns=["weekday", "timing"], prefix=["wd", "time"])

# sanity check
print(df_encoded.filter(like="wd_").columns)
print(df_encoded.filter(like="time_").columns)

Index(['wd_Friday', 'wd_Monday', 'wd_Saturday', 'wd_Sunday', 'wd_Thursday',
       'wd_Tuesday', 'wd_Wednesday'],
      dtype='object')
Index(['time_Morning', 'time_Night'], dtype='object')


In [5]:
df_encoded.head()

,time,use_house_overall,generated_solar,dishwasher,homeoffice,fridge,winecellar,garagedoor,barn,well,microwave,livingroom,temperature,humidity,visibility,pressure,windspeed,cloudcover,windbearing,precipprobability,furnace,kitchen,year,month,day,weekofyear,hour,minute,wd_Friday,wd_Monday,wd_Saturday,wd_Sunday,wd_Thursday,wd_Tuesday,wd_Wednesday,time_Morning,time_Night
0,2016-01-01 05:00:00,1.187276,0.038035,0.055087,0.091815,0.066464,0.019766,0.013373,0.097049,0.022193,0.008846,0.048255,33.010339,0.609728,9.905267,1013.323967,7.733644,0.355528,250.486667,0.000644,0.445069,0.011770,2016.0,1.0,1.966667,53.000000,11.3,29.5,False,False,True,False,False,False,False,True,False
1,2016-01-01 06:00:00,1.119695,0.080241,0.044929,0.131420,0.052407,0.019162,0.013601,0.052847,0.022646,0.014117,0.043144,18.775006,0.584022,9.916806,1024.161161,8.287394,0.089783,293.827222,0.000000,0.489006,0.010039,2016.0,1.0,4.450000,7.066667,11.7,29.5,False,True,False,False,False,False,False,False,True
2,2016-01-01 07:00:00,0.953479,0.074560,0.036512,0.100471,0.066452,0.017028,0.013425,0.065693,0.011260,0.011653,0.050291,24.236133,0.614706,9.929494,1027.502767,3.838150,0.037122,131.142778,0.000000,0.429768,0.010145,2016.0,1.0,6.966667,1.000000,11.3,29.5,False,False,False,False,True,False,False,True,False
3,2016-01-01 08:00:00,1.080323,0.013110,0.030701,0.097202,0.057517,0.021887,0.014500,0.099536,0.016839,0.007535,0.057834,39.956900,0.784944,8.501144,1012.871561,6.648811,0.488794,116.138889,0.147417,0.344875,0.011164,2016.0,1.0,9.450000,1.083333,11.7,29.5,False,False,True,False,False,False,False,False,True
4,2016-01-01 09:00:00,1.064834,0.046615,0.042406,0.099082,0.057820,0.015738,0.013374,0.034659,0.017822,0.009482,0.058562,24.463689,0.562878,9.119567,1010.960267,9.551461,0.165422,241.294444,0.014661,0.509327,0.009194,2016.0,1.0,11.966667,2.000000,11.3,29.5,False,False,False,False,False,True,False,True,False


In [6]:
df_encoded.to_csv("../data/interim/data_encoded.csv", index=False)